In [1]:
from imports import *
from exactDiagMethods import build_exact_diag_hamiltonian, perform_exact_diag
from globalMethods import  build_initial_states, output_results
from vqteMethods import hamiltonian_generation, perform_vqte

In [2]:

def main():
    # Initialize the simulation parameters
    N = 3
    gamma=0.5
    eps= 5
    mu_L=0.5
    mu_R=0.2
    T_L = 100
    T_R= 1
    beta_L=1/T_L
    beta_R=1/T_R
    F_L = 1/(np.exp(beta_L*(eps-mu_L))+1)
    F_R = 1/(np.exp(beta_R*(eps-mu_R))+1)
    J = 1
  
    time = 80.0
    dt = 1.0
    nt = int(time/dt)
    exact_diag_ham= build_exact_diag_hamiltonian(N,J, eps)
    print("Exact Ham", exact_diag_ham)

   
    ham_real, ham_imag = hamiltonian_generation(N, eps, gamma, F_L, F_R,J)
    print("Ham imag", ham_imag)
    print("Ham real", ham_real)

    vqte_init_state, exact_diag_init_state, ansatz, init_param_values = build_initial_states(ham_real,N)
    exact_diag_results, time_points = perform_exact_diag(gamma, F_L, F_R, dt, nt, exact_diag_init_state,exact_diag_ham,N,eps)

    print("VQTE initial state", vqte_init_state)
    print("Exact initial state:",exact_diag_init_state)
    vqte_results = perform_vqte(ham_real, ham_imag, vqte_init_state, dt, nt, ansatz, init_param_values,N)

    # # Output results
    output_results(vqte_results, exact_diag_results, time, nt, time_points)

In [3]:
main()

Exact Ham [[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  5.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  5.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j 10.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j  5.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j 10.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j 10.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j 15.+0.j]]
Ham imag SparsePauliOp(['XXIIII', 'YYIIII', 'IXXIII', 'IYYIII', 'IIXXII', 'IIYYII', 'IIIXXI', 'IIIYYI', 'IIIIXX', 'IIIIYY', 'IIIIII', 'ZIIIII', 'IIIIIZ'],
              coeffs=[-0.25      +0.j,  0.25      +0.j, -0.25      +0.j,  0.25      +0.j,
 -0.25      +0.j,  0.25      +0.j, -0.25      +0.j,  0.25      +0.j,
 -0.25      +0.j,  0.25      +0.j,  0.5       +0.j, -0.12781203+0.j,
 -0.24795936+0.j])
Ham real SparsePauliOp(['ZIIIII', 'IZIIII', 'IIZIII

ValueError: cannot reshape array of size 4096 into shape (64,1)